Day 7
====
In Camel Cards, you get a list of hands, and your goal is to order them based on the strength of each hand. A hand consists of five cards labeled one of A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, or 2. The relative strength of each card follows this order, where A is the highest and 2 is the lowest.

Every hand is exactly one type. From strongest to weakest, they are:

* Five of a kind, where all five cards have the same label: AAAAA
* Four of a kind, where four cards have the same label and one card has a different label: AA8AA
* Full house, where three cards have the same label, and the remaining two cards share a different label: 23332
* Three of a kind, where three cards have the same label, and the remaining two cards are each different from any other card in the hand: TTT98
* Two pair, where two cards share one label, two other cards share a second label, and the remaining card has a third label: 23432
* One pair, where two cards share one label, and the other three cards have a different label from the pair and each other: A23A4
* High card, where all cards' labels are distinct: 23456
* Hands are primarily ordered based on type; for example, every full house is stronger than any three of a kind.

If two hands have the same type, a second ordering rule takes effect. Start by comparing the first card in each hand. If these cards are different, the hand with the stronger first card is considered stronger. If the first card in each hand have the same label, however, then move on to considering the second card in each hand. If they differ, the hand with the higher second card wins; otherwise, continue with the third card in each hand, then the fourth, then the fifth.

So, 33332 and 2AAAA are both four of a kind hands, but 33332 is stronger because its first card is stronger. Similarly, 77888 and 77788 are both a full house, but 77888 is stronger because its third card is stronger (and both hands have the same first and second card).

Now, you can determine the total winnings of this set of hands by adding up the result of multiplying each hand's bid with its rank (765 * 1 + 220 * 2 + 28 * 3 + 684 * 4 + 483 * 5). So the total winnings in this example are 6440.

In [1]:
import pandas as pd
import re

#read in data
df = pd.read_table("7_data.txt", delimiter=" ",names=["Hand","Bid"])
df

,Hand,Bid
0,TT999,460
1,96T99,361
2,K264Q,948
3,K8TT3,68
4,T5JT5,730
...,...,...
995,57T79,415
996,37T48,563
997,T33T3,440
998,79422,721


In [2]:
#Make function to rank hands
#Score hands as:
# 6 5 of a kind 
# 5 4 of a kind
# 4 Full House
# 3 3 of a kind
# 2 2 pair
# 1 1 pair
# 0 High Card
def rankHand(string):
    matches=[]
    while len(string)>0:
        matches.append(len(re.findall(string[0],string)))
        string=string.replace(string[0], '')

    if (5 in matches):
        return 6
    if (4 in matches):
        return 5
    if (3 in matches) and (2 in matches):
        return 4
    if (3 in matches):
        return 3
    if (2 in matches) and len(matches)==3:
        return 2
    if (2 in matches):
        return 1
    return 0

df["handScore"]=df.apply(lambda x: rankHand(x.Hand),axis=1)
df

,Hand,Bid,handScore
0,TT999,460,4
1,96T99,361,3
2,K264Q,948,0
3,K8TT3,68,1
4,T5JT5,730,2
...,...,...,...
995,57T79,415,1
996,37T48,563,0
997,T33T3,440,4
998,79422,721,1


In [3]:
#Make function to give hands value if tied
#I think we can most easily do this by converting to hex...
def handValue(string):
    string=string.replace("T","a")
    string=string.replace("J","b")
    string=string.replace("Q","c")
    string=string.replace("K","d")
    string=string.replace("A","e")
    string="0x"+string
    return int(string,16)

df["handValue"]=df.apply(lambda x: handValue(x.Hand),axis=1)
df

,Hand,Bid,handScore,handValue
0,TT999,460,4,698777
1,96T99,361,3,617113
2,K264Q,948,0,861772
3,K8TT3,68,1,887459
4,T5JT5,730,2,678821
...,...,...,...,...
995,57T79,415,1,359033
996,37T48,563,0,227912
997,T33T3,440,4,668579
998,79422,721,1,496674


In [4]:
#Now to rank the hands and find the score
df["Rank"] = df[["handScore","handValue"]].apply(tuple,axis=1)\
             .rank(method='dense',ascending=True).astype(int)
df["Value"] = df.apply(lambda x: x.Rank*x.Bid,axis=1)
sum(df["Value"])

246795406

Part 2
------
Jacks are now Jokers, and worth less than a 2... but can change the value of the hand

In [6]:
#rewrite the function to score Jacks as 1 instead of b
def handValue2(string):
    string=string.replace("T","a")
    string=string.replace("J","1")
    string=string.replace("Q","c")
    string=string.replace("K","d")
    string=string.replace("A","e")
    string="0x"+string
    return int(string,16)

df["handValue2"]=df.apply(lambda x: handValue2(x.Hand),axis=1)
df

,Hand,Bid,handScore,handValue,Rank,Value,handValue2
0,TT999,460,4,698777,872,401120,698777
1,96T99,361,3,617113,728,262808,617113
2,K264Q,948,0,861772,162,153576,861772
3,K8TT3,68,1,887459,413,28084,887459
4,T5JT5,730,2,678821,555,405150,676261
...,...,...,...,...,...,...,...
995,57T79,415,1,359033,243,100845,359033
996,37T48,563,0,227912,16,9008,227912
997,T33T3,440,4,668579,864,380160,668579
998,79422,721,1,496674,281,202601,496674


In [16]:
#Make function to rank hands
#Score hands as:
# 6 5 of a kind 
# 5 4 of a kind
# 4 Full House
# 3 3 of a kind
# 2 2 pair
# 1 1 pair
# 0 High Card
def rankHand2(string):
    #I think jokers wild is pretty easy. We'll just count the jokers first, 
    #remove them from the had, and then ad them to the largest number of existing matches
    numJokers=len(re.findall("J",string))
    string=string.replace("J", '')
    
    matches=[]
    while len(string)>0:
        matches.append(len(re.findall(string[0],string)))
        string=string.replace(string[0], '')
    #edge case where hand is all J, matches will be []
    if numJokers==5:
        return 6
    
    matches.sort(reverse=True)    
    matches[0]=matches[0]+numJokers

    if (5 in matches):
        return 6
    if (4 in matches):
        return 5
    if (3 in matches) and (2 in matches):
        return 4
    if (3 in matches):
        return 3
    if (2 in matches) and len(matches)==3:
        return 2
    if (2 in matches):
        return 1
    return 0

df["handScore2"]=df.apply(lambda x: rankHand2(x.Hand),axis=1)
df


,Hand,Bid,handScore,handValue,Rank,Value,handValue2,handScore2
0,TT999,460,4,698777,872,401120,698777,4
1,96T99,361,3,617113,728,262808,617113,3
2,K264Q,948,0,861772,162,153576,861772,0
3,K8TT3,68,1,887459,413,28084,887459,1
4,T5JT5,730,2,678821,555,405150,676261,4
...,...,...,...,...,...,...,...,...
995,57T79,415,1,359033,243,100845,359033,1
996,37T48,563,0,227912,16,9008,227912,0
997,T33T3,440,4,668579,864,380160,668579,4
998,79422,721,1,496674,281,202601,496674,1


In [17]:
df["Rank2"] = df[["handScore2","handValue2"]].apply(tuple,axis=1)\
             .rank(method='dense',ascending=True).astype(int)
df["Value2"] = df.apply(lambda x: x.Rank2*x.Bid,axis=1)
sum(df["Value2"])

249356515